# Stent model

In [1]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

mapdl = launch_mapdl()

### Define Material Properties of 316L Stainless steel

In [2]:
mapdl.prep7()
mapdl.mptemp()
mapdl.mptemp(sloc='1', t1='0')
mapdl.mpdata(lab='EX', mat='1', c1='200e3')
mapdl.mpdata(lab='PRXY', mat='1', c1='0.3')
mapdl.mptemp()
mapdl.mptemp(sloc='1', t1='0')
mapdl.mpdata(lab='DENS', mat='1', c1='8000e-9')

PROPERTY TABLE DENS  MAT=      1  NUM. POINTS=  1
 SLOC=   1    0.8000000E-05

### Define Element Types

Beam elements have circular cross-sections as in the real S7 stent.
Use Beam189 to overcome convergence difficulty and to decrease computational time

In [3]:
#For straight line segments

mapdl.et(itype='1', ename='beam189')
mapdl.sectype(secid='1', type_='beam', subtype='csolid')
mapdl.secdata(val1=0.05)

#For arcs

mapdl.et(itype='2', ename='beam189')
mapdl.sectype(secid='2', type_='beam', subtype='csolid')
mapdl.secdata(val1=0.05)

SECTION ID NUMBER IS:            2
   BEAM SECTION TYPE IS:     Circular Solid  
   BEAM SECTION NAME IS:             
   COMPUTED BEAM SECTION DATA SUMMARY:
    Area                 = 0.78479E-02
    Iyy                  = 0.48986E-05
    Iyz                  =-0.66174E-22
    Izz                  = 0.48986E-05
    Warping Constant     = 0.46449E-40
    Torsion Constant     = 0.97972E-05
    Centroid Y           =-0.26983E-18
    Centroid Z           =-0.48569E-18
    Shear Center Y       = 0.36554E-20
    Shear Center Z       = 0.18437E-17
    Shear Correction-xy  = 0.85691    
    Shear Correction-yz  = 0.35124E-15
    Shear Correction-xz  = 0.85691    
                 
    Beam Section is offset to CENTROID of cross section

### Define 5 parameters Mooney-Rivlin hyperelastic artery material model.

In [4]:
c10=18.90e-3
c01=2.75e-3
c20=590.43e-3
c11=857.2e-3
nu1=0.49
dd=2*(1-2*nu1)/(c10+c01)

mapdl.tb(lab='hyper', mat='2', npts='5', tbopt='mooney')
mapdl.tbdata(stloc='1', c1='c10', c2='c01', c3='c20', c4='c11', c6='dd')

DATA FOR  HYPE  TABLE FOR MATERIAL   2 AT TEMPERATURE=  0.0000    
 LOC=  1 7.88861e-31     7.88861e-31     7.88861e-31     7.88861e-31     0.00000e+00     7.88861e-31

### Define linear elastic material model for stiff calcified plaque 

In [5]:
mapdl.mp(lab='EX', mat='3', c0='00219e3')
mapdl.mp(lab='NUXY', mat='3', c0='0.49')

MATERIAL          3     NUXY =  0.4900000

### Define Solid185 element type to mesh both the artery and plaque

In [6]:
mapdl.et(itype='9', ename='SOLID185') #For artery
mapdl.keyopt(itype='9', knum='6', value='1') #Use mixed u-P formulation to avoid locking
mapdl.keyopt(itype='9', knum='2', value='3') #Use Simplified Enhanced Strain method

mapdl.et(itype='16', ename='SOLID185') #For plaque
mapdl.keyopt(itype='16', knum='2', value='0') #Use B-bar

ELEMENT TYPE      16 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ    ROTX  ROTY  ROTZ
  THREE-DIMENSIONAL MODEL

Use force-distributed boundary constraints on 2 sides of artery wall to allow for radial expansion of tissue without rigid body motion.
Settings for MPC Surface-based, force-distributed contact on proximal plane parallel to x-y plane

In [7]:
mapdl.mat("2")
mapdl.r(nset="3")
mapdl.real(nset="3")
mapdl.et(itype="3", ename="170")
mapdl.et(itype="4", ename="174")
mapdl.keyopt(itype="4", knum="12", value="5")
mapdl.keyopt(itype="4", knum="4", value="1")
mapdl.keyopt(itype="4", knum="2", value="2")
mapdl.keyopt(itype="3", knum="2", value="1")
mapdl.keyopt(itype="3", knum="4", value="111111")
mapdl.type(itype="3")

mapdl.mat("2")
mapdl.r(nset="4")
mapdl.real(nset="4")
mapdl.et(itype="5", ename="170")
mapdl.et(itype="6", ename="174")
mapdl.keyopt(itype="6", knum="12", value="5")
mapdl.keyopt(itype="6", knum="4", value="1")
mapdl.keyopt(itype="6", knum="2", value="2")
mapdl.keyopt(itype="5", knum="2", value="1")
mapdl.keyopt(itype="5", knum="4", value="111111")
mapdl.type(itype="5")

ELEMENT TYPE SET TO         5

Settings for standard contact between stent and inner plaque wall contact surface

In [8]:
mapdl.mp(lab="MU", mat="1", c0="0")
mapdl.mat("1")
mapdl.mp(lab="EMIS", mat="1", c0="7.88860905221e-31")
mapdl.r(nset="6")
mapdl.real(nset="6")
mapdl.et(itype="10", ename="170")
mapdl.et(itype="11", ename="177")
mapdl.r(nset="6", r3="1.0", r4="1.0", r5="0")
mapdl.rmore(r9="1.0E20", r10="0.0", r11="1.0")
mapdl.rmore(r7="0.0", r8="0", r9="1.0", r10="0.05", r11="1.0", r12="0.5")
mapdl.rmore(r7="0", r8="1.0", r9="1.0", r10="0.0")
mapdl.keyopt(itype="11", knum="5", value="0")
mapdl.keyopt(itype="11", knum="7", value="1")
mapdl.keyopt(itype="11", knum="8", value="0")
mapdl.keyopt(itype="11", knum="9", value="0")
mapdl.keyopt(itype="11", knum="10", value="2")
mapdl.keyopt(itype="11", knum="11", value="0")
mapdl.keyopt(itype="11", knum="12", value="0")
mapdl.keyopt(itype="11", knum="2", value="3")
mapdl.keyopt(itype="10", knum="5", value="0")


ELEMENT TYPE      10 IS TARGE170     3-D TARGET SEGMENT          
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ    ROTX  ROTY  ROTZ
  THREE-DIMENSIONAL MODEL

Settings for MPC based, force-distributed constraint on proximal stent nodes

In [9]:
mapdl.mat("1")
mapdl.r(nset="7")
mapdl.real(nset="7")
mapdl.et(itype="12", ename="170")
mapdl.et(itype="13", ename="175")
mapdl.keyopt(itype="13", knum="12", value="5")
mapdl.keyopt(itype="13", knum="4", value="1")
mapdl.keyopt(itype="13", knum="2", value="2")
mapdl.keyopt(itype="12", knum="2", value="1")
mapdl.keyopt(itype="12", knum="4", value="111111")
mapdl.type(itype="12")

ELEMENT TYPE SET TO        12

Settings for MPC based, force-distributed constraint on distal stent nodes

In [10]:
mapdl.mat("1")
mapdl.r(nset="8")
mapdl.real(nset="8")
mapdl.et(itype="14", ename="170")
mapdl.et(itype="15", ename="175")
mapdl.keyopt(itype="15", knum="12", value="5")
mapdl.keyopt(itype="15", knum="4", value="1")
mapdl.keyopt(itype="15", knum="2", value="2")
mapdl.keyopt(itype="14", knum="2", value="1")
mapdl.keyopt(itype="14", knum="4", value="111111")
mapdl.type(itype="14")

ELEMENT TYPE SET TO        14

Read the geometry file

In [12]:
mapdl.cdread(option="db", fname="stent", ext="cdb")
mapdl.allsel(labt="all")
mapdl.finish()

***** ROUTINE COMPLETED *****  CP =         0.984

In [13]:
mapdl.run("/solu")
mapdl.antype(antype="0")
mapdl.nlgeom(key="on")

LARGE DEFORMATION ANALYSIS

Apply Load Step 1: Balloon angioplasty of the artery to expand it past the radius of the stent - IGNORE STENT

In [14]:
mapdl.nsubst(nsbstp="20", nsbmx="20")
mapdl.nropt(option1="full")
mapdl.cncheck(option="auto")
mapdl.esel(type_="s", item="type", vmin="11")
mapdl.cm(cname="contact2", entity="elem")
mapdl.ekill(elem="contact2") # Kill contact elements in stent-plaque contact 
                             # pair so that the stent is ignored in the first
                             # loadstep

mapdl.nsel(type_="s", item="loc", comp="x", vmin="0", vmax="0.01e-3")
mapdl.nsel(type_="r", item="loc", comp="y", vmin="0", vmax="0.01e-3")
mapdl.d(node="all", lab="all")
mapdl.allsel()

mapdl.sf(nlist="load", lab="pres", value="10e-2") #Apply 0.1 Pa/mm^2 pressure to inner plaque wall
mapdl.allsel()
mapdl.nldiag(label="cont", key="iter")
mapdl.solve()
mapdl.save()



ALL CURRENT MAPDL DATA WRITTEN TO FILE NAME= file.db
  FOR POSSIBLE RESUME FROM THIS POINT

Apply Load Step 2: Reactivate contact between stent and plaque

In [16]:
mapdl.ealive(elem="contact2")
mapdl.allsel()

mapdl.nsubst(nsbstp="2", nsbmx="2")
mapdl.save()
mapdl.solve()

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =     100.391   TIME= 12:47:05
 Present time 0 is less than or equal to the previous time.  Time will   
 default to 2.                                                           

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   12:47:05  NOV 16, 2022 CP=    100.734

                                                                               



                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     2
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  2.0000    
   AUTOMATIC TIME STEPPING . . . . . . . . . . . .    ON
      INITIAL NUMBER OF SUBSTEPS . . . . . . . . .     2
      MAXIMUM NUMBER OF SUBSTEPS . . . . . . . . .     2
      MINIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
   MAXIMUM NUMBER OF EQUILIBRIUM ITERATIONS. .

Apply Load Step 3

In [17]:
mapdl.nsubst(nsbstp="1", nsbmx="1", nsbmn="1")
mapdl.solve()

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =     123.219   TIME= 12:48:20
 Present time 0 is less than or equal to the previous time.  Time will   
 default to 3.                                                           

 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2022 R2          22.2     ***
 Ansys Mechanical Enterprise                       
 00000000  VERSION=WINDOWS x64   12:48:21  NOV 16, 2022 CP=    123.469

                                                                               



                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     3
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  3.0000    
   AUTOMATIC TIME STEPPING . . . . . . . . . . . .    ON
      INITIAL NUMBER OF SUBSTEPS . . . . . . . . .     1
      MAXIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
      MINIMUM NUMBER OF SUBSTEPS . . . . . . . . .     1
   MAXIMUM NUMBER OF EQUILIBRIUM ITERATIONS. .

Apply Load Step 4: Apply blood pressure (13.3 kPa) load to inner wall of plaque and allow the stent to act as a scaffold

In [18]:
mapdl.nsubst(nsbstp="300", nsbmx="3000", nsbmn="30")
mapdl.sf(nlist="load", lab="pres", value="13", value2="3e-3")
mapdl.allsel()

SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW

Apply stabilization with energy option

In [19]:
mapdl.stabilize(key="const", method="energy", value="0.1")

NONLINEAR STABILIZATION CONTROL:
 KEY=CONS   METHOD=ENER   VALUE= 0.10000       SUBSTPOPT=NO     FORCELIMIT= 0.20000

In [20]:
mapdl.solve()
mapdl.save()
mapdl.finish()

In [ ]:
mapdl.exit()